https://ddingmin00.tistory.com/entry/Selenium-%ED%8E%98%EC%9D%B4%EC%A7%80%EB%A5%BC-%EB%82%B4%EB%A0%A4%EA%B0%80%EB%A9%B0-%EB%AC%B4%ED%95%9C-%ED%81%AC%EB%A1%A4%EB%A7%81-%ED%95%98%EA%B8%B0

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# import time

In [ ]:
# option 만들기
# options = Options()
# options.add_experimental_option("detach", True)
# service = Service(ChromeDriverManager().install())
# driver=webdriver.Chrome(service=service, options=options)

In [ ]:
# actions = driver.find_element(By.CSS_SELECTOR, 'body')
# actions.send_keys(Keys.END)

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
import time

import pandas as pd
from tqdm import tqdm

driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
url = "https://wtable.co.kr/recipes"

menu_selector = "#__next > div > main > div > div > div > section:nth-child(3) > section > div > div > div > a > div > p.RecipeItemstyle__Title-sc-1gt7vz8-3.hpYiJK"
level_selector = "#__next > div > main > div > div > div > section:nth-child(3) > section > div > div > div > a:nth-child(37) > div > div > div.RecipeItemstyle__Time-sc-1gt7vz8-6.RecipeItemstyle__Level-sc-1gt7vz8-7.gvzBms > span"
links_selector = "#__next > div > main > div > div > div > section:nth-child(3) > section > div > div > div > a"

driver.get(url)

actions = driver.find_element(By.CSS_SELECTOR, 'body')

def crawling(count):
    menus, levels, links = [], [], []
    
    for _ in range(count):
        for i in driver.find_elements(By.CSS_SELECTOR, menu_selector):
            menus.append(i.text)
        for i in driver.find_elements(By.CSS_SELECTOR, level_selector):
            levels.append(i.text)
        for i in driver.find_elements(By.CSS_SELECTOR, links_selector):
            links.append(i.get_attribute("href"))
        actions.send_keys(Keys.END)
        time.sleep(.8)
    
    return menus, levels, links


menus, levels, links = crawling(3)

print("가져온 상품 수: ", len(menus))
for level, level, link in zip(menus, levels, links):
    print("상품: ", menu)
    print("난이도: ", level)
    print("주소: ", link)
    print()

가져온 상품 수:  240
상품:  치즈볼
난이도:  초급
주소:  https://wtable.co.kr/recipes/ddmTZR6jBFNzbXdzZJ7AjLCu?location=recipe_home

상품:  치즈볼
난이도:  초급
주소:  https://wtable.co.kr/recipes/ggpjdY7Xs2AyjCCwPohGhabt?location=recipe_home

상품:  치즈볼
난이도:  초급
주소:  https://wtable.co.kr/recipes/5cEMJ2c75qBM6x7iQqnCWASv?location=recipe_home



In [22]:
df_1 = pd.DataFrame({
    '메뉴': menus,
    '난이도': level,
    '주소': links
})

df_1['난이도'].unique()

array(['초급'], dtype=object)

In [3]:
import pandas as pd

def get_recipe_info(link):
    driver.get(link)
    time.sleep(.5)
    
    # 소요시간 가져오기
    time_selector = "#__next > div > main > div.token__Component-sc-1o2h3sm-0.jjTxDH > section > div.RecipeDetailstyle__Summary-q7sykd-5.iObVZL > div.RecipeDetailstyle__Level-q7sykd-6.RecipeDetailstyle__Time-q7sykd-7.cEiCHw > p:nth-child(2)"
    try:
        cook_time = driver.find_element(By.CSS_SELECTOR, time_selector).text.strip()
    except:
        cook_time = "N/A"
    
    # 재료 가져오기
    ##__next > div > main > div.token__Component-sc-1o2h3sm-0.jjTxDH > div:nth-child(3) > div > ul > li:nth-child(1) > ul > li:nth-child(3) > div > div:nth-child(1)
    ing_selector = "#__next > div > main > div.token__Component-sc-1o2h3sm-0.jjTxDH > div:nth-child(3) > div > ul > li > ul > li > div > div:nth-child(1)"
    ings = []
    for ing in driver.find_elements(By.CSS_SELECTOR, ing_selector):
        ings.append(ing.text)
    
    return cook_time, ings

results = []
for link in links:
    cook_time, ings = get_recipe_info(link)
    results.append([cook_time, ings])

df = pd.DataFrame(results, columns=["소요시간", "재료"])
df.head()

,소요시간,재료
0,30분,"[찹쌀가루, 밀가루(중력분), 설탕, 베이킹파우더, 우유, 소금, 모짜렐라 치즈, ..."
1,30분,"[칼솟, 파프리카(노랑, 빨강), 볶은 아몬드, 토마토소스, 깐 마늘, 올리브오일,..."
2,20분,"[꼬시래기, 치킨스테이크(또는 닭가슴살), 샐러드 채소, 된장, 매실액, 현미식초]"
3,20분,"[꼬시래기, 깻잎, 양파, 오이, 통깨, 식초, 소금, 고추장, 고춧가루, 식초, ..."
4,20분,"[꼬시래기, 시판용 냉면육수, 오이, 삶은 달걀, 쌈무, 통깨, 소금, 식초, 고추..."


In [4]:
all = pd.concat([df_1, df], axis=1)
all

,상품,난이도,주소,소요시간,재료
0,치즈볼,초급,https://wtable.co.kr/recipes/ddmTZR6jBFNzbXdzZ...,30분,"[찹쌀가루, 밀가루(중력분), 설탕, 베이킹파우더, 우유, 소금, 모짜렐라 치즈, ..."
1,칼솟타다,초급,https://wtable.co.kr/recipes/ggpjdY7Xs2AyjCCwP...,30분,"[칼솟, 파프리카(노랑, 빨강), 볶은 아몬드, 토마토소스, 깐 마늘, 올리브오일,..."
